`03-pdf-rag.ipynb`

In [4]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()


True

In [ ]:
llm.invoke('testing')

In [ ]:
# 1. 문서 로딩(Loader)
from langchain_community.document_loaders import PyMuPDFLoader

# 2. 문서 나누기(Split)
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 3. 문서 변환하기(Embed) => GPT를 사용하기 때문에 OpenAI 에서 제공하는 Embedding 사용
from langchain_openai import OpenAIEmbeddings

# 4. Embedding 저장하기 (Vector Store)
from langchain_community.vectorstores import FAISS

In [ ]:
%pip install pymupdf

In [ ]:
# 1. Load
loader = PyMuPDFLoader('./data.pdf')
docs = loader.load()
print(f'문서 페이지 수: {len(docs)}')

In [ ]:
# 2. Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)
print(f'분할된 청크의 수: {len(split_docs)}')

In [ ]:
# 3. Embed Setting
embeddings = OpenAIEmbeddings()

In [ ]:
%pip install faiss-cpu

In [ ]:
# 4. Vectorstore
vectorstore = FAISS.from_documents(documents=split_docs, embedding=embeddings)

In [ ]:
# 5. 검색(retrieve) => vectorstore 내장 검색기
# 6. 프롬프트 생성(prompt)
from langchain_core.prompts import PromptTemplate

# 7. LLM 
from langchain_openai import ChatOpenAI

# 8. Chain

In [ ]:
# 5. Retrieve
retriever = vectorstore.as_retriever()

In [ ]:
# 6. Prompting

text = '''
넌 질문-답변을 도와주는 AI 비서야.
아래 제공되는 Context를 통해서 사용자 Question에 대해 답을 해줘야해.

Context에는 직접적으로 없어도, 추론하거나 계산할 수 있는 답변은 최대한 만들어 봐.
그리고 관련 내용을 말할때는, pdf 몇 페이지이서 찾은 내용인지도 말해줘.

답은 적절히 \n를 통해 문단을 나눠줘 한국어로 만들어 줘. 
# Question:
{question}

# Context:
{context}


# Answer:
'''

prompt = PromptTemplate.from_template(text)

In [ ]:
# 7. LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
# 8. Chain
chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

In [ ]:
ans = chain.invoke('삼성전자 관련 소식들을 다 가져와')

In [ ]:
print(ans)

## ConversationChain (대화 내용 기억하는 체인)
- Without RAG

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a Kind Good Chatbot '),
        ('placeholder', '{chat_history}'),
        ('human', '{input}')
    ]
)

history = InMemoryChatMessageHistory()
parser = StrOutputParser()

chain = prompt | llm | parser

wrapped_chain = RunnableWithMessageHistory(chain, lambda h: history)

In [ ]:
ans = wrapped_chain.invoke(
    {'input': 'Hello. My name is Yu.'},
    config={'configurable': {'session_id': 715211}}
)

In [ ]:
ans2 = wrapped_chain.invoke(
    {'input': 'Any nickname recommendation?'},
    config={'configurable': {'session_id': 715211}}
)

In [ ]:
ans3 = wrapped_chain.invoke(
    {'input': 'What is my name?'},
    config={'configurable': {'session_id': 715211}}
)

In [ ]:
ans3

## `ConversationalRetrievalChain`
- Question-Answering with RAG

In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# ConversationalQA
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# 1. Load
loader = PyMuPDFLoader('./data.pdf')
docs = loader.load()
print(f'문서 페이지 수: {len(docs)}')

# 2. Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)
print(f'분할된 청크의 수: {len(split_docs)}')

# 3. Embed Setting
embeddings = OpenAIEmbeddings()

# 4. Vectorstore
vectorstore = FAISS.from_documents(documents=split_docs, embedding=embeddings)

question_prompt_template = '''
    Given a chat history and the latest user question
    which might reference context in the chat history,
    formulate a standalone question which can be understood
    without the chat history. Do NOT answer the question,
    just reformulate it if needed and otherwise return it as is.
'''

question_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', question_prompt_template),
        ('placeholder', '{chat_history}'),
        ('human', '{input}')
    ]
)

# 5. Retrieve
retriever = create_history_aware_retriever(
    llm, vectorstore.as_retriever(), question_prompt,
)

# 6. Prompting
prompt = '''
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. Use three sentences maximum and keep the answer concise.
Answer in Korean.
# Context:
{context}
'''

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

qa_chain = create_stuff_documents_chain(llm, qa_prompt)
conv_qa_chain = create_retrieval_chain(
    retriever, qa_chain
)

문서 페이지 수: 23
분할된 청크의 수: 72


In [5]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []

while True:
    user_msg = input('물어봐: ')
    ans = conv_qa_chain.invoke(
        {
            'input': user_msg,
            'chat_history': chat_history
        }
    )
    chat_history += [
        HumanMessage(user_msg),
        AIMessage(ans['answer'])
    ]
    print(user_msg)
    print(ans['answer'])
    
    

삼성 관련 자료 다 가져와
삼성 가우스는 온디바이스에서 작동 가능한 생성 AI 모델로, 언어, 코드, 이미지의 3개 모델로 구성되어 있습니다. 이 모델은 안전한 데이터를 통해 학습되어 개인정보를 침해하지 않으며, 사용자 정보 유출 위험이 없는 장점을 가지고 있습니다. 또한, 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획이며, 이를 통해 삼성 스마트폰이 경쟁력을 갖출 것으로 예상됩니다.
방금 말한거에서 AI 이름만 가져와
삼성 가우스

안녕하세요! 궁금한 것이 있으시면 무엇이든 물어보세요. 도와드리겠습니다.

언제든지 도움이 필요하시면 또 문의해 주세요. 좋은 하루 되세요!
